In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

In [2]:
from config import TabulatorConfig
cfg = TabulatorConfig.from_file('2013_minneapolis_park_bottoms_up_config.json')
cfg

TabulatorConfig(tabulator_version='TEST', output_settings=TabulatorOutputConfig(contest_name='2013 Minneapolis Park Board', output_directory='output', contest_date='', contest_jurisdiction='Minneapolis', contest_office='Park and Recreation Commissioner', tabulate_by_precinct=False, generate_cdf_json=False), cvr_file_sources=[CvrSourceConfig(file_path='2013_minneapolis_park_bottoms_up_cvr.xlsx', first_vote_column_index=2, first_vote_row_index=2, id_column_index=None, precinct_column_index=1, provider='ess', treat_blank_as_undeclared_write_in=False, overvote_label='overvote', undervote_label='undervote', undeclared_write_in_label='UWI')], candidates=[C(ANNIE YOUNG), C(CASPER HILL), C(HASHIM YONIS), C(ISHMAEL ISRAEL), C(JASON STONE), C(JOHN ERWIN), C(MARY LYNN MCPHERSON), C(MEG FORNEY), C(STEVE BARLAND), C(TOM NORDYKE)], rules=TabulatorRules(tiebreak_mode=<TiebreakMode.RANDOM: 'random'>, overvote_rule=<OvervoteRule.ALWAYS_SKIP_TO_NEXT_RANK: 'alwaysSkipToNextRank'>, winner_election_mode=<W

In [3]:
src_cfg = cfg.cvr_file_sources[0]
df = src_cfg.load_df()
df

,Precinct,1ST CHOICE PARK AND RECREATION COMMISSIONER AT LARGE MINNEAPOLIS,2ND CHOICE PARK AND RECREATION COMMISSIONER AT LARGE MINNEAPOLIS,3RD CHOICE PARK AND RECREATION COMMISSIONER AT LARGE MINNEAPOLIS,COUNT
0,MINNEAPOLIS W-11 P-05,JOHN ERWIN,TOM NORDYKE,ANNIE YOUNG,1
1,MINNEAPOLIS W-10 P-03A,MEG FORNEY,undervote,undervote,1
2,MINNEAPOLIS W-1 P-03,JOHN ERWIN,MARY LYNN MCPHERSON,MEG FORNEY,1
3,MINNEAPOLIS W-2 P-01,STEVE BARLAND,MEG FORNEY,CASPER HILL,1
4,MINNEAPOLIS W-7 P-04D,undervote,undervote,undervote,1
...,...,...,...,...,...
80096,MINNEAPOLIS W-8 P-07,undervote,undervote,undervote,1
80097,MINNEAPOLIS W-1 P-05,JASON STONE,undervote,undervote,1
80098,MINNEAPOLIS W-10 P-01,undervote,undervote,undervote,1
80099,MINNEAPOLIS W-1 P-03,ANNIE YOUNG,JOHN ERWIN,undervote,1


In [18]:
from tabulate import tabulate, CvrStatus
from fractions import Fraction
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

CHOICES = df.columns[src_cfg.first_vote_column_index-1:][:cfg.rules.max_rankings_allowed]
df.rename(inplace=True, columns=dict(zip(CHOICES, [c[:3] for c in CHOICES])))
CHOICES =  [c[:3] for c in CHOICES]
CANDIDATES = [c.name for c in cfg.candidates]

tabulate(df, CHOICES, divide=Fraction)



INFO:tabulate:Round 1. FIGHT!
DEBUG:tabulate:Progressing 21199 votes, {'undervote': 20761, 'UWI': 337, 'overvote': 101}
DEBUG:tabulate:Progressing 20984 votes, {'undervote': 20849, 'UWI': 121, 'overvote': 14}
DEBUG:tabulate:Progressing 20927 votes, {'undervote': 20811, 'UWI': 105, 'overvote': 11}
INFO:tabulate:Threshold: 59174/3 votes
INFO:tabulate:Gosh darn losers: ['ANNIE YOUNG', 'CASPER HILL', 'HASHIM YONIS', 'ISHMAEL ISRAEL', 'JASON STONE']
DEBUG:tabulate:Progressing 18079 votes, {'JASON STONE': 6448, 'HASHIM YONIS': 4019, 'ISHMAEL ISRAEL': 3874, 'undervote': 2138, 'CASPER HILL': 1477, 'ANNIE YOUNG': 100, 'UWI': 19, 'overvote': 4}
DEBUG:tabulate:Progressing 11858 votes, {'undervote': 6793, 'ANNIE YOUNG': 1927, 'ISHMAEL ISRAEL': 1065, 'HASHIM YONIS': 967, 'JASON STONE': 613, 'CASPER HILL': 428, 'UWI': 58, 'overvote': 7}
DEBUG:tabulate:Progressing 6564 votes, {'undervote': 4910, 'ANNIE YOUNG': 450, 'HASHIM YONIS': 445, 'JASON STONE': 288, 'ISHMAEL ISRAEL': 225, 'CASPER HILL': 191, 'U

TypeError: both arguments should be Rational instances

In [11]:
cvr_status = CvrStatus(df, CHOICES)

pd.factorize(df[CHOICES])

ValueError: could not broadcast input array from shape (80101,3) into shape (80101,)

In [8]:
CHOICES = df.columns[src_cfg.first_vote_column_index-1:][:cfg.rules.max_rankings_allowed]
df.rename(inplace=True, columns=dict(zip(CHOICES, [c[:3] for c in CHOICES])))
CHOICES =  [c[:3] for c in CHOICES]
CANDIDATES = [c.name for c in cfg.candidates]

CHOICE_LOOKUP = (df[CHOICES].join(pd.Series([np.nan], name='fallback'))).to_numpy()
NP_DF_RANGE = np.arange(len(df))

status = pd.DataFrame(index=df.index)
status['cur_choice_i'] = 0
status['cur_choice'] = df[CHOICES[0]]
status['vote_remaining'] = 1.
status['remains'] = True
def next_choice_if(mask):
    status.cur_choice_i += mask
    status.cur_choice = CHOICE_LOOKUP[NP_DF_RANGE, status.cur_choice_i]

num_winners = 2
winners = []
losers = []
while len(winners) < num_winners:
    # drop_votes; naively skip until valid or exhausted
    invalid_vote = status.cur_choice.isin(['undervote', 'overvote', 'UWI'] + winners + losers)
    while sum(invalid_vote) > 0:
        print(f"Dropping {sum(invalid_vote)} invalid votes, {status.cur_choice[invalid_vote].value_counts().to_dict()}")
        next_choice_if(invalid_vote)
        invalid_vote = status.cur_choice.isin(['undervote', 'overvote', 'UWI'] + winners + losers)

    # status.cur_choice means exhausted
    valid_vote = status.cur_choice.notnull() & ~invalid_vote
    threshold = (1/(num_winners - len(winners) + 1)) * sum(valid_vote)
    totals = status[valid_vote].groupby('cur_choice')['vote_remaining'].sum()
    if round_winners := totals.index[totals > threshold].to_list():
        print(f"Ayyoo winners: {round_winners}")
        winners += round_winners
        # do in groupby or somethin
        for winner in round_winners:
            extra_ratio = (totals[winner] - threshold) / totals[winner]
            is_winner = status.cur_choice == winner
            status.loc[is_winner, 'vote_remaining'] *= extra_ratio
            print(f"Redistribute {winner}'s remaining {sum(status.loc[is_winner, 'vote_remaining'])} votes")
            next_choice_if(is_winner)
    else:
        round_losers = totals.index[totals.sort_values().cumsum() < threshold].to_list()
        losers += round_losers
        print(f"Gosh darn losers: {round_losers}")
        for loser in round_losers:
            is_loser = status.cur_choice == loser
            print(f"Redistribute {loser}'s remaining {sum(status.loc[is_loser, 'vote_remaining'])} votes")
            next_choice_if(is_loser)



# idx, cols = pd.factorize(df['names'])

Dropping 21199 invalid votes, {'undervote': 20761, 'UWI': 337, 'overvote': 101}
Dropping 20984 invalid votes, {'undervote': 20849, 'UWI': 121, 'overvote': 14}
Dropping 20927 invalid votes, {'undervote': 20811, 'UWI': 105, 'overvote': 11}
Gosh darn losers: ['ANNIE YOUNG', 'CASPER HILL', 'HASHIM YONIS', 'ISHMAEL ISRAEL', 'JASON STONE']
Redistribute ANNIE YOUNG's remaining 9302.0 votes
Redistribute CASPER HILL's remaining 1477.0 votes
Redistribute HASHIM YONIS's remaining 4056.0 votes
Redistribute ISHMAEL ISRAEL's remaining 4510.0 votes
Redistribute JASON STONE's remaining 7049.0 votes
Dropping 11141 invalid votes, {'undervote': 6927, 'ANNIE YOUNG': 2151, 'HASHIM YONIS': 986, 'CASPER HILL': 485, 'ISHMAEL ISRAEL': 452, 'UWI': 74, 'JASON STONE': 55, 'overvote': 11}
Dropping 8268 invalid votes, {'undervote': 6914, 'HASHIM YONIS': 389, 'ANNIE YOUNG': 326, 'JASON STONE': 245, 'ISHMAEL ISRAEL': 202, 'CASPER HILL': 134, 'UWI': 54, 'overvote': 4}
Ayyoo winners: ['JOHN ERWIN']
Redistribute JOHN ER

In [16]:
from fractions import Fraction

CHOICES = df.columns[src_cfg.first_vote_column_index-1:][:cfg.rules.max_rankings_allowed]
df.rename(inplace=True, columns=dict(zip(CHOICES, [c[:3] for c in CHOICES])))
CHOICES =  [c[:3] for c in CHOICES]
CANDIDATES = [c.name for c in cfg.candidates]


CHOICE_LOOKUP = (df[CHOICES].join(pd.Series([np.nan], name='fallback'))).to_numpy()
NP_DF_RANGE = np.arange(len(df))

max_denom = 0

def repr_f(num):
    global max_denom
    match num:
        case Fraction(numerator=num, denominator=denom) if denom != 1:
            if denom > max_denom:
                max_denom = denom
                print(f"New max denominator: {denom}")
            return f"~approx~{num/denom:.2f}"
        case _:
            return repr(num)

status = pd.DataFrame(index=df.index)
status['cur_choice_i'] = 0
status['cur_choice'] = df[CHOICES[0]]
status['vote_remaining'] = 1
status['remains'] = True
def next_choice_if(mask):
    status.cur_choice_i += mask
    status.cur_choice = CHOICE_LOOKUP[NP_DF_RANGE, status.cur_choice_i]

num_winners = 2
winners = []
losers = []
while len(winners) < num_winners:
    # drop_votes; naively skip until valid or exhausted
    invalid_vote = status.cur_choice.isin(['undervote', 'overvote', 'UWI'] + winners + losers)
    while sum(invalid_vote) > 0:
        print(f"Dropping {sum(invalid_vote)} invalid votes, {status.cur_choice[invalid_vote].value_counts().to_dict()}")
        next_choice_if(invalid_vote)
        invalid_vote = status.cur_choice.isin(['undervote', 'overvote', 'UWI'] + winners + losers)

    # status.cur_choice means exhausted
    valid_vote = status.cur_choice.notnull() & ~invalid_vote

    threshold = Fraction(sum(valid_vote),(num_winners - len(winners) + 1)) 

    totals = status[valid_vote].groupby('cur_choice')['vote_remaining'].sum()
    if round_winners := totals.index[totals > threshold].to_list():
        print(f"Ayyoo winners: {round_winners}")
        winners += round_winners
        # do in groupby or somethin
        for winner in round_winners:
            extra_ratio = (totals[winner] - threshold) / totals[winner]
            is_winner = status.cur_choice == winner
            status.loc[is_winner, 'vote_remaining'] *= extra_ratio
            print(f"Redistribute winner {winner}'s remaining {repr_f(sum(status.loc[is_winner, 'vote_remaining']))} votes")
            next_choice_if(is_winner)
    else:
        round_losers = totals.index[totals.sort_values().cumsum() < threshold].to_list()
        losers += round_losers
        print(f"Gosh darn losers: {round_losers}")
        for loser in round_losers:
            is_loser = status.cur_choice == loser
            print(f"Redistribute loser {loser}'s remaining {repr_f(sum(status.loc[is_loser, 'vote_remaining']))} votes")
            next_choice_if(is_loser)



# idx, cols = pd.factorize(df['names'])

Dropping 21199 invalid votes, {'undervote': 20761, 'UWI': 337, 'overvote': 101}
Dropping 20984 invalid votes, {'undervote': 20849, 'UWI': 121, 'overvote': 14}
Dropping 20927 invalid votes, {'undervote': 20811, 'UWI': 105, 'overvote': 11}
Gosh darn losers: ['ANNIE YOUNG', 'CASPER HILL', 'HASHIM YONIS', 'ISHMAEL ISRAEL', 'JASON STONE']
Redistribute loser ANNIE YOUNG's remaining 9302 votes
Redistribute loser CASPER HILL's remaining 1477 votes
Redistribute loser HASHIM YONIS's remaining 4056 votes
Redistribute loser ISHMAEL ISRAEL's remaining 4510 votes
Redistribute loser JASON STONE's remaining 7049 votes
Dropping 11141 invalid votes, {'undervote': 6927, 'ANNIE YOUNG': 2151, 'HASHIM YONIS': 986, 'CASPER HILL': 485, 'ISHMAEL ISRAEL': 452, 'UWI': 74, 'JASON STONE': 55, 'overvote': 11}
Dropping 8268 invalid votes, {'undervote': 6914, 'HASHIM YONIS': 389, 'ANNIE YOUNG': 326, 'JASON STONE': 245, 'ISHMAEL ISRAEL': 202, 'CASPER HILL': 134, 'UWI': 54, 'overvote': 4}
Ayyoo winners: ['JOHN ERWIN']
